In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 24.7 MB/s eta 0:00:00:00:0100:01


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig, LogitsProcessor, LogitsProcessorList
import os
import torch
import json

In [3]:
# import sys
# sys.path.append('/kaggle/input/nlp-project-data/')

In [4]:
from utils import match_x
from tqdm import tqdm

In [5]:
data = match_x(sentences_file='sentences.json', identifiers_file='identifiers.json')

In [6]:
texts = []
for _, x in data.items():
    texts.append([])
    for sentence in x:
        texts[-1].append(sentence['x'])

texts = texts[:100]

In [7]:
model_path = "llama-3/transformers/8b-chat-hf/1"

bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_8bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            # bnb_4bit_use_double_quant=True,
#             llm_int8_enable_fp32_cpu_offload=True
        )

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             device_map="auto", 
                                             low_cpu_mem_usage=True,
                                             quantization_config=bnb_config,
                                            )

Unused kwargs: ['bnb_8bit_quant_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [9]:
max_steps = 0

def create_prompts(texts):
    prompts = []
    global max_steps
    for text in texts:
        max_steps = max(max_steps, len(text))
        prompt = "### Instructions: \n"
        # prompt += "The following sentences represent steps of a recipe but the order are shuffled.:\n\n"
        prompt += "The following sentences represent steps of a recipe but the order are shuffled.\
                   Plesse reorder and then concatenate them to match the correct sequence for the recipe:\n\n"
        for step in text:
            prompt += f"{step}\n"
        # prompt += f"\nReorder them to match the correct sequence for the recipe.\
        #             Please only output the correct order in numerical numbers sequence, splitted by comma and only include {len(text)} numbers :\n\n"
        
        prompt += "\n### Correct Order Sequences:"
        prompts.append(prompt)
    print(f"max steps: {max_steps}")
    return prompts

In [13]:
def reorder_recipe_steps(prompts):
    results = []
    for prompt in tqdm(prompts):
        
        inputs  = tokenizer(prompt, return_tensors="pt")
        
        outputs = model.generate(
            **inputs,
            max_new_tokens = 512,
            pad_token_id=tokenizer.eos_token_id,
        )
        res = tokenizer.decode(outputs[0], skip_special_tokens=True)
        try:
            results.append(res.split("### Correct Order Sequences:")[1])
        except:
            results.append('')
    return results

results = reorder_recipe_steps(prompts)

  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
100%|██████████| 100/100 [43:16<00:00, 25.96s/it]


In [14]:
answers = {'answers': results}
with open("answers.json", "w", encoding="utf-8") as file:
    json.dump(answers, file)